In [ ]:
# many-to-many RNN

# 품사 태깅, 개체명 인식 등에 사용됨

In [23]:
# 문자 단위 RNN(Char RNN)
# RNN의 입출력 단위가 문자인 경우

# 문자 시퀀스 apple을 입력받으면 pple!를 출력하는 RNN을 구현

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# 훈련 데이터 전처리하기
input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str+label_str)))  # 문자 집합 만들기
print(char_vocab)
vocab_size = len(char_vocab)
print ('문자 집합의 크기 : {}'.format(vocab_size))

# 하이퍼파라미터 정의
# 원-핫 벡터를 사용할 것이므로 입력의 크기는 문자 집합 크기와 같아야 함
input_size = vocab_size  # 입력 크기 = 문자 집합의 크기
hidden_size = 5
output_size = 5
learning_rate = 0.1

char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 문자 집합에 고유한 정수를 부여
print(char_to_index)
index_to_char = {}  # 예측 결과를 다시 문자로 보기 위한 것
for key, value in char_to_index.items():
    index_to_char[value] = key
print(index_to_char)

# 입력 데이터와 레이블 데이터 매핑
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data)
print(y_data)

# 배치 차원 추가: nn.RNN()은 3차원 텐서를 입력으로 받음
x_data  = [x_data]
y_data = [y_data]
print(x_data)
print(y_data)

# 입력 시퀀스의 각 문자들을 one-hot벡터로 바꿔줌
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

# 데이터 텐서로 변환
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

# RNN 모델 구현하기
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size, output_size, bias=True) # 출력층
    
    def forward(self,x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x
    
net = Net(input_size, hidden_size, output_size)
outputs = net(X)
print(outputs.shape) # 3차원 텐서 (배치 차원, 시점(timesteps), 출력)
# 정확도를 측정할 때는 이를 모두 펼쳐서 계산
# view를 사용하여 배치 차원과 시점 차원을 하나로 만들기
print(outputs.view(-1, input_size).shape) # 2차원 텐서로 변환

print(Y.shape)
print(Y.view(-1).shape)

# 옵티마이저와 손실 함수 정의
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

# 학습진행
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, input_size), Y.view(-1))  # Batch 차원 제거
    loss.backward()
    optimizer.step()
    
    result = outputs.data.numpy().argmax(axis=2) # 최종 예측값인 각 time-step 별 5차원 벡터에 대해서 가장 높은 값의 인덱스를 선택
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

['!', 'a', 'e', 'l', 'p']
문자 집합의 크기 : 5
{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}
{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}
[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]
[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]
[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]
훈련 데이터의 크기 : torch.Size([1, 5, 5])
레이블의 크기 : torch.Size([1, 5])
torch.Size([1, 5, 5])
torch.Size([5, 5])
torch.Size([1, 5])
torch.Size([5])
0 loss:  1.7598316669464111 prediction:  [[3 3 3 3 3]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  lllll
1 loss:  1.4910593032836914 prediction:  [[2 3 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  elle!
2 loss:  1.290828824043274 prediction:  [[2 3 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  elle!
3 loss:  1.075850248336792 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
4 loss:  0.870538055896759 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!


81 loss:  0.0013560022925958037 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
82 loss:  0.00133845629170537 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
83 loss:  0.001320861978456378 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
84 loss:  0.001303266966715455 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
85 loss:  0.0012856714893132448 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
86 loss:  0.0012680275831371546 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
87 loss:  0.0012502879835665226 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
88 loss:  0.0012324047274887562 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
89 loss:  0.0012144260108470917 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
90 loss:  0.00119620817

In [33]:
# 문자 단위 RNN(Char RNN) - 더 많은 데이터

import torch
import torch.nn as nn
import torch.optim as optim

sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

char_set = list(set(sentence))  # 중복을 제거한 문자 집합 생성
char_dic = {c:i for i,c in enumerate(char_set)}  # 각 문자에 정수 인코딩
print(char_dic)  # 공백도 하나의 원소로 침
dic_size = len(char_dic)

# 하이퍼파라이터 설정
hidden_size = dic_size
sequence_length = 10  # 10개 문자 단위로 잘라 샘플을 만들것임
learning_rate = 0.1

# 샘플 만들기
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i+sequence_length]
    y_str = sentence[i+1:i+sequence_length+1]
    # print(i, x_str, '->', y_str)
    
    x_data.append([char_dic[c] for c in x_str])
    y_data.append([char_dic[c] for c in y_str])
    
print(x_data[0])  # if you wan에 해당됨.
print(y_data[0])  # f you want에 해당됨.

x_one_hot = [np.eye(dic_size)[x] for x in x_data]  # 입력 시퀀스 원-핫 인코딩
X = torch.FloatTensor(x_one_hot)  # 텐서로 변환
Y = torch.LongTensor(y_data)
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))
print(X[0])  
print(Y[0])  #  f you want에 해당

class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers): 
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x
    
net = Net(dic_size, hidden_size, 2)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X) # (170, 10, 25)크기의 텐서를 매 epoch마다 모델의 입력으로 사용
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1)) # [1700, 25]와 [1700]크기 비교
    loss.backward()
    optimizer.step()

    # results의 텐서 크기는 (170, 10)
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오지만
            predict_str += ''.join([char_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += char_set[result[-1]]

    print(predict_str)

{'h': 0, 'b': 1, 't': 2, 'o': 3, 'k': 4, 'u': 5, "'": 6, 'a': 7, 'y': 8, ',': 9, ' ': 10, 'n': 11, 'w': 12, 'p': 13, '.': 14, 'f': 15, 'm': 16, 'r': 17, 'g': 18, 'd': 19, 'c': 20, 'e': 21, 'i': 22, 'l': 23, 's': 24}
[22, 15, 10, 8, 3, 5, 10, 12, 7, 11]
[15, 10, 8, 3, 5, 10, 12, 7, 11, 2]
훈련 데이터의 크기 : torch.Size([170, 10, 25])
레이블의 크기 : torch.Size([170, 10])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 

l you want to build a ship, don't arum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
l you want to build a ship, don't drum up people together to collect wood and don't assign them tosks and work, but rather teach them to long for the endless immensity of the sea.
l you want to build a ship, don't drum up people together to collect wood and don't assign them tosks and work, but rather teach them to long for the endless immensity of the sea.
l you want to build a ship, don't drum up people together to collect wood and don't dssign them tosks and work, but rather teach them to long for the endless immensity of the sea.
l you want to build a ship, don't drum up people together to collect wood and don't assign them tosks and work, but rather toach them to long for the endless immensity of the sea.
l you want to build a ship, don't arum up people together to collect wood and don't assign them tosk

In [42]:
# 단어 단위 RNN - 임베딩 사용

import torch
import torch.nn as nn
import torch.optim as optim

# 임의의 문장
# 'Repeat is the best medicine for'을 입력받으면 'is the best medicine for memory'를 출력하는 RNN 만들기
sentence = "Repeat is the best medicine for memory".split()
print(sentence)

# 단어장 만들기
vocab = list(set(sentence))
print(vocab)
word2index = {tnk:i for i,tnk in enumerate(vocab, 1)}  # 단어에 고유한 정수 부여
word2index['<unk>'] = 0  # 모르는 단어를 의미하는 UNK 토큰 추가
print(word2index)

index2word = {v:k for k,v in word2index.items()}
print(index2word)

# 데이터의 각 단어를 정수로 인코딩
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence]  # 각 문자를 정수로 변환
    input_seq, label_seq = encoded[:-1], encoded[1:]  # 입력 시퀀스, 레이블 시퀀스
    
    input_seq = torch.LongTensor(input_seq).unsqueeze(0)  # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0)  # 배치 차원 추가
    
    return input_seq, label_seq

X, Y = build_data(sentence, word2index)
print(X)  # Repeat is the best medicine for을 의미
print(Y)  # is the best medicine for memory을 의미

class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size) # 출력은 원-핫 벡터의 크기를 가져야함. 또는 단어 집합의 크기만큼 가져야함.

    def forward(self, x):
        
        # 1. 임베딩 층
        # 크기변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        
        # 2. RNN 층
        # 크기변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        
        # 3. 최종 출력층
        # 크기변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
        output = self.linear(output)
        
        # 4. view를 통해서 배치 차원 제거
        # 크기변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기)
        
        return output.view(-1, output.size(2))

    
# 하이퍼 파라미터
vocab_size = len(word2index)  # 단어장의 크기는 임베딩 층, 최종 출력층에 사용
input_size = 5  # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20  # RNN의 은닉층 크기

model = Net(vocab_size, input_size, hidden_size, batch_first=True)
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함
optimizer = optim.Adam(params=model.parameters())

# 수치화된 데이터를 단어로 전환하는 함수
decode = lambda y: [index2word.get(x) for x in y]

for step in range(201):

    optimizer.zero_grad()
    output = model(X)

    loss = loss_function(output, Y.view(-1))
    loss.backward()
    optimizer.step()
  
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

['Repeat', 'is', 'the', 'best', 'medicine', 'for', 'memory']
['the', 'memory', 'Repeat', 'medicine', 'is', 'best', 'for']
{'the': 1, 'memory': 2, 'Repeat': 3, 'medicine': 4, 'is': 5, 'best': 6, 'for': 7, '<unk>': 0}
{1: 'the', 2: 'memory', 3: 'Repeat', 4: 'medicine', 5: 'is', 6: 'best', 7: 'for', 0: '<unk>'}
tensor([[3, 5, 1, 6, 4, 7]])
tensor([[5, 1, 6, 4, 7, 2]])
[01/201] 2.1572 
Repeat <unk> <unk> memory <unk> Repeat <unk>

[41/201] 1.4645 
Repeat for the best medicine for memory

[81/201] 0.8481 
Repeat is the best medicine for memory

[121/201] 0.4352 
Repeat is the best medicine for memory

[161/201] 0.2322 
Repeat is the best medicine for memory

[201/201] 0.1422 
Repeat is the best medicine for memory

